# Reverse Mode Implementation (High Level)

#### In designing our implementation we have focused on 

1. Expressibility
2. Efficiency
3. Ease of Use

## Expressibility

Given that Dotua is meant as a partial NumPy replacement, it is essential that users are able to express any mathematical function with Dotua that they could with NumPy.  To this end Dotua offers the following function support with reverse mode automatic differentation

- Basic functions: addition, subtraction, multiplication, division, exponentiation, and negation
- Trigonometric functions: sine, cosine, and tangent
- Inverse trigonometric functions: arcsine, arccosine, arctangent, 
- Hyperbolic functions: 
- Logarithms of any base
- Roots of arbitrary degree
- Logistic functions

In [3]:
# Complex function demo
rad_init = rAutoDiff()

vals = [7, 11, 13, 17, 23, 29]
a, b, c, d, f, h = rad.create_rscalar(vals)

func = op.arccosh(a) / (op.log(c) ** (op.arcsin(b ** -2))) + \
       op.cos((op.tan(h + a) ** op.log(d * h, base=2)) / (op.sin(f) * op.cos(f))

func.eval()
func.partial(a)
func.partial(b)
func.partial(c)
func.partial(d)
func.partial(f)
func.partial(h)

Note: Embed Wolfram Alpha calculation of derivative

## Eficiency

- Compared with forward mode, fewer operations necessary to compute a gradient of a single function
- O(n) vs. O(m)

## Ease of Use

- Similar to NumPy definitions
- Extensive function support

# Reverse Mode Implementation (Low Level)

#### In designing our implementation we have focused on 

1. rScalar
2. rOperator
3. rAutoDiff

## rScalar



## rOperator



## rAutoDiff

